In [7]:
import json
import sqlite3
from pathlib import Path
from urllib.parse import urljoin

import pandas as pd
from slugify import slugify
from sqlite_utils import Database

In [8]:
df_nsw = pd.read_csv("srnsw-indexes.csv")

In [9]:
def make_download_link(title):
    """
    Create a link to download the CSV file from GitHub
    """
    filename = "{}.csv".format(slugify(title))
    url = urljoin(
        "https://raw.githubusercontent.com/wragge/srnsw-indexes/master/data/", filename
    )
    return url


df_nsw["download"] = df_nsw["title"].apply(lambda x: make_download_link(x))

In [10]:
df_nsw

,id,index,title,more_info_url,status,url,download
0,9,Surname|Firstname,Assisted Immigrants,https://www.records.nsw.gov.au/archives/collec...,Fully digitised,https://www.records.nsw.gov.au/searchhits_noco...,https://raw.githubusercontent.com/wragge/srnsw...
1,8,Surname|Firstname,Australian Railway Supply Detachment,https://www.records.nsw.gov.au/archives/collec...,Fully digitised,https://www.records.nsw.gov.au/searchhits_noco...,https://raw.githubusercontent.com/wragge/srnsw...
2,59,Surname|FirstName,Bankruptcy Index,https://www.records.nsw.gov.au/archives/collec...,Not digitised,https://www.records.nsw.gov.au/searchhits_noco...,https://raw.githubusercontent.com/wragge/srnsw...
3,11,Surname|FirstName,"Bench of Magistrates cases, 1788-1820",https://www.records.nsw.gov.au/archives/collec...,Not digitised,https://www.records.nsw.gov.au/searchhits_noco...,https://raw.githubusercontent.com/wragge/srnsw...
4,91,Surname|First names,Botanic Gardens and Government Domains Employe...,https://www.records.nsw.gov.au/archives/collec...,Not digitised,https://www.records.nsw.gov.au/searchhits_noco...,https://raw.githubusercontent.com/wragge/srnsw...
5,90,Surname|First Names,Bubonic Plague Index,https://www.records.nsw.gov.au/archives/collec...,Fully digitised,https://www.records.nsw.gov.au/searchhits_noco...,https://raw.githubusercontent.com/wragge/srnsw...
6,74,Surname|FirstName|Remarks,Child Care and Protection,https://www.records.nsw.gov.au/archives/collec...,Not digitised,https://www.records.nsw.gov.au/searchhits_noco...,https://raw.githubusercontent.com/wragge/srnsw...
7,41,Surname|Firstname,Closer and Soldier Settlement Transfer Files,https://www.records.nsw.gov.au/archives/collec...,Not digitised,https://www.records.nsw.gov.au/searchhits_noco...,https://raw.githubusercontent.com/wragge/srnsw...
8,62,Surname|Firstname|Surname of Transferee|Firstn...,"Closer Settlement Transfer Registers, NRS 8082",https://www.records.nsw.gov.au/archives/collec...,Not digitised,https://www.records.nsw.gov.au/searchhits_noco...,https://raw.githubusercontent.com/wragge/srnsw...
9,65,Surname|Firstname|Alias,Convict Index,https://www.records.nsw.gov.au/archives/collec...,Not digitised,https://www.records.nsw.gov.au/searchhits_noco...,https://raw.githubusercontent.com/wragge/srnsw...


In [11]:
metadata = json.loads(Path("metadata.json").read_bytes())
tables = metadata["databases"]["nsw-state-archives"]["tables"]
db = Database(sqlite3.connect("nsw-state-archives.db"))
for csv in df_nsw.itertuples():
    print(csv.title)
    tables[slugify(csv.title)] = {
        "title": csv.title,
        "source_url": csv.download,
        "about_url": csv.more_info_url,
        "license": "CC BY 4.0",
        "searchmode": "raw",
    }
    table = db[slugify(csv.title)]
    df_csv = pd.read_csv(csv.download, keep_default_na=False, low_memory=False)
    table.insert_all(df_csv.to_dict("records"))
    cols_to_index = csv.index.split("|")
    # print(cols_to_index)
    table.enable_fts(cols_to_index)

metadata["databases"]["nsw-state-archives"]["tables"] = tables
with Path("metadata.json").open("w") as json_file:
    json_file.write(json.dumps(metadata))

Assisted Immigrants
Australian Railway Supply Detachment
Bankruptcy Index
Bench of Magistrates cases, 1788-1820
Botanic Gardens and Government Domains Employees Index 
Bubonic Plague Index
Child Care and Protection
Closer and Soldier Settlement Transfer Files
Closer Settlement Transfer Registers, NRS 8082
Convict Index
Convicts Applications to Marry 1825-51
Coroners Inquests 1796-1824
Court of Civil Jurisdiction index
Crew (and Passenger) Lists, 1828-1841
Criminal Court Records index 1788-1833
Criminal Indictments, 1863-1919
CSreLand
Deceased Estates
Depasturing Licenses
Devonshire Street Cemetery Reinterment Index, 1901
Divorce Index
Early Convict Index
FieldBooks
Government Asylums for the Infirm and Destitute
Governor’s Court Case Papers, 1815-1824
Index on Occupants on Aboriginal Reserves, 1875 to 1904
Index to 1841 Census
Index to Closer Settlement Promotion
Index to convict exiles, 1846-50
Index to Court of Claims
Index to Deposition Registers
Index to Early Probate Records
Index